In [1]:
!pip install git+git://github.com/louisabraham/python3-midi@master


  Cloning git://github.com/louisabraham/python3-midi (to revision master) to /tmp/pip-req-build-9cc24el_
  Running setup.py bdist_wheel for midi ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-nkknksbe/wheels/30/20/89/3147155a68fcb47992316b826500cedfde8e3d7b18333a4560
Successfully built midi
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install keras

tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorboard 1.8.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import midi
import numpy as np


lowerBound = 24
upperBound = 102
span = upperBound-lowerBound
num_rest = 0
num_play_hold = 1
num_play_artic = 2

def midiToNoteStateMatrix(midifile, squash=True):
    pattern = midi.read_midifile(midifile)

    timeleft = [track[0].tick for track in pattern]

    posns = [0 for track in pattern]

    statematrix = []
    time = 0

    state = [[0,0] for x in range(span)]
    statematrix.append(state)
    condition = True
    while condition:
        if time % (pattern.resolution / 4) == (pattern.resolution / 8):
            # Crossed a note boundary. Create a new state, defaulting to holding notes
            oldstate = state
            state = [[oldstate[x][0],0] for x in range(span)]
            statematrix.append(state)
        for i in range(len(timeleft)): #For each track
            if not condition:
                break
            while timeleft[i] == 0:
                track = pattern[i]
                pos = posns[i]

                evt = track[pos]
                if isinstance(evt, midi.NoteEvent):
                    if (evt.pitch < lowerBound) or (evt.pitch >= upperBound):
                        pass
                        # print "Note {} at time {} out of bounds (ignoring)".format(evt.pitch, time)
                    else:
                        if isinstance(evt, midi.NoteOffEvent) or evt.velocity == 0:
                            state[evt.pitch-lowerBound] = [0, 0]
                        else:
                            state[evt.pitch-lowerBound] = [1, 1]
                elif isinstance(evt, midi.TimeSignatureEvent):
                    if evt.numerator not in (2, 4):
                        # We don't want to worry about non-4 time signatures. Bail early!
                        # print "Found time signature event {}. Bailing!".format(evt)
                        out =  statematrix
                        condition = False
                        break
                try:
                    timeleft[i] = track[pos + 1].tick
                    posns[i] += 1
                except IndexError:
                    timeleft[i] = None

            if timeleft[i] is not None:
                timeleft[i] -= 1

        if all(t is None for t in timeleft):
            break

        time += 1

    S = np.array(statematrix)
    #statematrix = np.hstack((S[:, :, 0], S[:, :, 1]))
    #statematrix = np.asarray(statematrix).tolist()
    
    return statematrix



def noteStateMatrixToMidi(statematrix, name="example"):
    statematrix = np.array(statematrix)
    if not len(statematrix.shape) == 3:
        statematrix = np.dstack((statematrix[:, :span], statematrix[:, span:]))
    statematrix = np.asarray(statematrix)
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    
    span = upperBound-lowerBound
    tickscale = 55
    
    lastcmdtime = 0
    prevstate = [[0,0] for x in range(span)]
    for time, state in enumerate(statematrix + [prevstate[:]]):  
        offNotes = []
        onNotes = []
        for i in range(span):
            n = state[i]
            p = prevstate[i]
            if p[0] == 1:
                if n[0] == 0:
                    offNotes.append(i)
                elif n[1] == 1:
                    offNotes.append(i)
                    onNotes.append(i)
            elif n[0] == 1:
                onNotes.append(i)
        for note in offNotes:
            track.append(midi.NoteOffEvent(tick=(time-lastcmdtime)*tickscale, pitch=note+lowerBound))
            lastcmdtime = time
        for note in onNotes:
            track.append(midi.NoteOnEvent(tick=(time-lastcmdtime)*tickscale, velocity=40, pitch=note+lowerBound))
            lastcmdtime = time
            
        prevstate = state
    
    eot = midi.EndOfTrackEvent(tick=1)
    track.append(eot)

    midi.write_midifile("{}.mid".format(name), pattern)

In [6]:
lista_state_matrixs.append(state_m)

NameError: name 'lista_state_matrixs' is not defined

In [7]:
import os.path as op
import os
limit = 20
lista_state_matrixs= []
lista_archivos = os.listdir("midi_band")
#Poner a Guitar primero
for ind, midi_path in enumerate(lista_archivos):
    if "Guitar" in midi_path:
        position=lista_archivos[0]
        lista_archivos[0]=midi_path
        lista_archivos[ind]=position
#lista_archivos = ["/home/inquisidor/Desktop/incognito/proyecto-beatles/midis_por_instrumento/Organ/The_Beatles_-_She_Loves_You_Organ.mid"]
#lista_archivos= ["midi_band/The_Beatles_-_Hey_Jude_Bass.mid"]
state_guitar = midiToNoteStateMatrix(op.join('midi_band',lista_archivos[0]), squash=True)
state_vacio=np.zeros(np.array(state_guitar).shape)
indicador=5
instrumentos=["Guitar","Bass","Reed"]
while len(lista_state_matrixs) < 3:
    for ind,midi_path in enumerate(lista_archivos):
        f_path = op.join('midi_band',midi_path)
        if instrumentos[len(lista_state_matrixs)] in f_path:
            indicador=ind
    if indicador==5:
        lista_state_matrixs.append(state_vacio)
    else:
        state_m = midiToNoteStateMatrix(op.join('midi_band',lista_archivos[indicador]), squash=True)
        lista_state_matrixs.append(state_m)
    indicador=5
lista_state_matrixs = np.vstack(lista_state_matrixs) 

state_band=lista_state_matrixs

f_path= "midi_band/The_Beatles_-_Hey_Jude_Guitar.mid"



print(np.array(lista_state_matrixs).shape)

state_guitar = midiToNoteStateMatrix(op.join('midi_band',lista_archivos[0]), squash=True)
print(np.array_equal(lista_state_matrixs[1345*2:1345*3],state_guitar))

(4035, 78, 2)
False


In [8]:
state_band.shape
out=[]
n = state_band.shape[0] // 3

for i in range(n):
    g=state_band[n*0+i]
    bass=state_band[n*1 + i]
    reed=state_band[n*2 + i]
    
    out.append(np.concatenate((g,bass,reed), axis=None))
    
out_v_stack = np.vstack(out)


rec = out_v_stack.reshape(-1,78,2)


In [9]:
guitars = rec[0::3]
bajo = rec[1::3]
redd = rec[2::3]

t=np.vstack([guitars,bajo,redd])

np.array_equal(state_band,t)

True

In [10]:
out_v_stack.shape, 78*3*2

((1345, 468), 468)

In [11]:
import keras
from keras.datasets import mnist
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model

notas=78*2
tiempos = 10

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(notas*tiempos,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim*3, activation='relu')(input_img)
encoded2 = Dense(encoding_dim*2, activation='relu')(encoded)
encoded3 = Dense(encoding_dim, activation='relu')(encoded2)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim*2*3, activation='relu')(encoded3)
decoded1 = Dense(encoding_dim*3*3, activation='relu')(decoded)
decoded2 = Dense(notas*tiempos*3, activation='sigmoid')(decoded1)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded2)
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded3)
 
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer1 = autoencoder.layers[-3](encoded_input)
decoder_layer2 = autoencoder.layers[-2](decoder_layer1)
decoder_layer3 = autoencoder.layers[-1](decoder_layer2)
# create the decoder model
decoder = Model(encoded_input, decoder_layer3)
 
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')


data_song = np.array(state_guitar)
data_train = []
for i in range(data_song.shape[0]-tiempos):
    pieza = data_song[i : i+tiempos] # tiempos x notas
    data_train.append(pieza.flatten()) # tiempos*notas

data_train=np.vstack(data_train) # N x (tiempos * notas)
    
data_song = np.array(out_v_stack)
data_train_out = []
for i in range(data_song.shape[0]-tiempos):
    pieza = data_song[i : i+tiempos] # tiempos x notas
    data_train_out.append(pieza.flatten()) # tiempos*notas

data_train_out=np.vstack(data_train_out) # N x (tiempos * notas) * 3 instrumentos
    
# aa=np.asarray(a[1::],dtype='float32')
#x_train = x_train.astype('float32') / 255.
#x_test = x_test.astype('float32') / 255.
#x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
#x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

#aa=np.transpose(aa, (0, 2, 1))
# x_train=aa.reshape(-1,78*2*10)

autoencoder.fit(data_train, data_train_out,
                epochs=500,
                batch_size=100,
                shuffle=False,
                validation_data=(data_train, data_train_out))

/home/inquisidor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 1335 samples, validate on 1335 samples
Epoch 1/500
1335/1335 [==============================] - 1s 633us/step - loss: 0.6928 - val_loss: 0.6923
Epoch 2/500
1335/1335 [==============================] - 0s 72us/step - loss: 0.6920 - val_loss: 0.6915
Epoch 3/500
1335/1335 [==============================] - 0s 69us/step - loss: 0.6912 - val_loss: 0.6907
Epoch 4/500
1335/1335 [==============================] - 0s 67us/step - loss: 0.6903 - val_loss: 0.6898
Epoch 5/500
1335/1335 [==============================] - 0s 65us/step - loss: 0.6894 - val_loss: 0.6888
Epoch 6/500
1335/1335 [==============================] - 0s 63us/step - loss: 0.6883 - val_loss: 0.6876
Epoch 7/500
1335/1335 [==============================] - 0s 65us/step - loss: 0.6870 - val_loss: 0.6862
Epoch 8/500
1335/1335 [==============================] - 0s 62us/step - loss: 0.6853 - val_loss: 0.6839
Epoch 9/500
1335/1335 [==============================] - 0s 65us/step - loss: 0.6823 - val_loss: 0.6793
Epoch 10/500
13

1335/1335 [==============================] - 0s 74us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 79/500
1335/1335 [==============================] - 0s 72us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 80/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 81/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 82/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 83/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 84/500
1335/1335 [==============================] - 0s 72us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 85/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0389
Epoch 86/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0371 - val_loss: 0.0388
Epoch 87/500
1335/1335 [==============================] - 0s 73us/st

1335/1335 [==============================] - 0s 75us/step - loss: 0.0368 - val_loss: 0.0383
Epoch 156/500
1335/1335 [==============================] - 0s 72us/step - loss: 0.0368 - val_loss: 0.0383
Epoch 157/500
1335/1335 [==============================] - 0s 72us/step - loss: 0.0368 - val_loss: 0.0383
Epoch 158/500
1335/1335 [==============================] - 0s 71us/step - loss: 0.0368 - val_loss: 0.0383
Epoch 159/500
1335/1335 [==============================] - 0s 71us/step - loss: 0.0367 - val_loss: 0.0383
Epoch 160/500
1335/1335 [==============================] - 0s 74us/step - loss: 0.0367 - val_loss: 0.0383
Epoch 161/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0367 - val_loss: 0.0383
Epoch 162/500
1335/1335 [==============================] - 0s 74us/step - loss: 0.0367 - val_loss: 0.0383
Epoch 163/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0367 - val_loss: 0.0382
Epoch 164/500
1335/1335 [==============================] - 0

1335/1335 [==============================] - 0s 76us/step - loss: 0.0323 - val_loss: 0.0341
Epoch 233/500
1335/1335 [==============================] - 0s 74us/step - loss: 0.0322 - val_loss: 0.0339
Epoch 234/500
1335/1335 [==============================] - 0s 76us/step - loss: 0.0321 - val_loss: 0.0338
Epoch 235/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0320 - val_loss: 0.0337
Epoch 236/500
1335/1335 [==============================] - 0s 74us/step - loss: 0.0319 - val_loss: 0.0336
Epoch 237/500
1335/1335 [==============================] - 0s 78us/step - loss: 0.0318 - val_loss: 0.0335
Epoch 238/500
1335/1335 [==============================] - 0s 78us/step - loss: 0.0317 - val_loss: 0.0333
Epoch 239/500
1335/1335 [==============================] - 0s 80us/step - loss: 0.0316 - val_loss: 0.0332
Epoch 240/500
1335/1335 [==============================] - 0s 80us/step - loss: 0.0315 - val_loss: 0.0331
Epoch 241/500
1335/1335 [==============================] - 0

1335/1335 [==============================] - 0s 63us/step - loss: 0.0262 - val_loss: 0.0268
Epoch 310/500
1335/1335 [==============================] - 0s 65us/step - loss: 0.0262 - val_loss: 0.0268
Epoch 311/500
1335/1335 [==============================] - 0s 69us/step - loss: 0.0262 - val_loss: 0.0267
Epoch 312/500
1335/1335 [==============================] - 0s 88us/step - loss: 0.0262 - val_loss: 0.0267
Epoch 313/500
1335/1335 [==============================] - 0s 88us/step - loss: 0.0261 - val_loss: 0.0267
Epoch 314/500
1335/1335 [==============================] - 0s 87us/step - loss: 0.0261 - val_loss: 0.0267
Epoch 315/500
1335/1335 [==============================] - 0s 90us/step - loss: 0.0261 - val_loss: 0.0266
Epoch 316/500
1335/1335 [==============================] - 0s 89us/step - loss: 0.0261 - val_loss: 0.0266
Epoch 317/500
1335/1335 [==============================] - 0s 89us/step - loss: 0.0260 - val_loss: 0.0266
Epoch 318/500
1335/1335 [==============================] - 0

1335/1335 [==============================] - 0s 69us/step - loss: 0.0253 - val_loss: 0.0257
Epoch 387/500
1335/1335 [==============================] - 0s 67us/step - loss: 0.0253 - val_loss: 0.0257
Epoch 388/500
1335/1335 [==============================] - 0s 69us/step - loss: 0.0253 - val_loss: 0.0257
Epoch 389/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0253 - val_loss: 0.0257
Epoch 390/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0253 - val_loss: 0.0257
Epoch 391/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0252 - val_loss: 0.0257
Epoch 392/500
1335/1335 [==============================] - 0s 69us/step - loss: 0.0252 - val_loss: 0.0256
Epoch 393/500
1335/1335 [==============================] - 0s 67us/step - loss: 0.0252 - val_loss: 0.0256
Epoch 394/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0252 - val_loss: 0.0256
Epoch 395/500
1335/1335 [==============================] - 0

1335/1335 [==============================] - 0s 67us/step - loss: 0.0247 - val_loss: 0.0251
Epoch 464/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0247 - val_loss: 0.0251
Epoch 465/500
1335/1335 [==============================] - 0s 68us/step - loss: 0.0247 - val_loss: 0.0251
Epoch 466/500
1335/1335 [==============================] - 0s 66us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 467/500
1335/1335 [==============================] - 0s 62us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 468/500
1335/1335 [==============================] - 0s 64us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 469/500
1335/1335 [==============================] - 0s 66us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 470/500
1335/1335 [==============================] - 0s 70us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 471/500
1335/1335 [==============================] - 0s 73us/step - loss: 0.0247 - val_loss: 0.0250
Epoch 472/500
1335/1335 [==============================] - 0

In [10]:
all([np.all(aa[-1].reshape(-1) == x_train[-1])for i in range(len(aa))] )

NameError: name 'aa' is not defined

In [ ]:
t.reshape(10*78*2)[-10:]

In [ ]:
t[-1,-5:,:]

In [ ]:
t=np.random.rand(10,78,2)
t

In [12]:
78*2*1601
78*2*10
# bb.shape
from datetime import datetime

In [13]:
def prob(x,a=0.5):
  return sum(sum(x>a)) / (x.shape[0]*x.shape[1])


for i in range(5):

    encoded_imgs = encoder.predict(data_train)
    decoded_imgs = decoder.predict(encoded_imgs)
    generated=np.random.rand(500,32)
    decoded_generated= decoder.predict(generated)
    probabilidad=0.55
    while prob(data_train_out)>prob(decoded_generated,probabilidad):
        probabilidad=probabilidad-0.0001
    print(probabilidad)
    probabilidad=(probabilidad+0.5)/2
    bb=(decoded_generated > probabilidad).astype(np.int)

    # bb=bb.reshape(-1,78,2)

    def get_data_L(data): # N x tiempos*notas
        # asume que los datos estan ordenadaso en N por ventana consecutivas
        reconst = []
        for ind,row in enumerate(data):
            ventana=row.reshape(tiempos,78,2)
            if ind == 0:
                for t_step in ventana:
                    reconst.append(t_step.reshape(1,78,2))
            else:
                reconst.append(ventana[-1].reshape(1,78,2))
        return np.vstack(reconst).tolist()

    nombre = 'Organ'
    now = datetime.now().strftime("%Y-%m-%d-%H_%M:%S")
    bb=get_data_L(bb)
    # bb=np.transpose(bb, (2, 0, 1))  
    # bb=bb.tolist()
    #a.shape
    #x_train.shape
    #bb.shape
    noteStateMatrixToMidi(bb, name="example_a_{0}_{1}_{2}".format(nombre,now,i))

0.4604000000000099


ValueError: cannot reshape array of size 4680 into shape (10,78,2)

In [33]:
from datetime import datetime
def prob(x,a=0.5):
  return sum(sum(x>a)) / (x.shape[0]*x.shape[1])


for i in range(5):

    encoded_imgs = encoder.predict(data_train)
    decoded_imgs = decoder.predict(encoded_imgs)
    generated=np.random.rand(500,32)
    decoded_generated= decoder.predict(generated)
  #  probabilidad=0.55
   # while prob(data_train_out)>prob(decoded_generated,probabilidad):
  #       probabilidad=probabilidad-0.0001
 #   print(probabilidad)
 #   probabilidad=(probabilidad+0.5)/2
    bb=(decoded_generated > 0.4).astype(np.int)
    
    


    def get_data_L(data): # N x tiempos*notas*3
        out={'guitar' : [],'bass' : [],'reed' : [] }
        
        # asume que los datos estan ordenadaso en N por ventana consecutivas
        for ind,row in enumerate(data):
            
            # vector de tiempos * notas * 3
            ###Considerar hacer transpose###
        #    [N, Tiempo, Nota, 3 ]
            
            ventana=row.reshape(tiempos,78,2,3) #todo
            
            lista = [(0,'guitar'),(1,'bass'),(2,'reed')]
            
            for ind_instrumento,instrumento in lista:
                
                canal = ventana[:,:,:,ind_instrumento]  # tiempos x notas x 2 
                #rint(canal.shape)
                if ind == 0:
                    for t_step in canal:
                        out[instrumento].append(t_step.reshape(1,78,2))
                else:
                    out[instrumento].append(canal[-1].reshape(1,78,2))
                    
                    
        for k in out:
            out[k] = np.vstack(out[k]).tolist()
            
        return out
    
    now = datetime.now().strftime("%Y-%m-%d-%H_%M:%S")
    bb=get_data_L(bb)

    for k in bb:
        state_m = bb[k]
        noteStateMatrixToMidi(state_m, name="midi_{0}_{1}_{2}".format(k,now,i))

In [138]:
decoded_generated

array([[0.13954696, 0.14649247, 0.13768177, ..., 0.14192909, 0.14061064,
        0.15962036],
       [0.13366795, 0.13605843, 0.14284606, ..., 0.13285421, 0.1395875 ,
        0.15221009],
       [0.19334488, 0.1852351 , 0.19558714, ..., 0.18736039, 0.1870777 ,
        0.20340383],
       ...,
       [0.19290812, 0.19696087, 0.18685536, ..., 0.18366106, 0.17778866,
        0.20366493],
       [0.2013438 , 0.19937603, 0.20069438, ..., 0.19691512, 0.2036324 ,
        0.21993266],
       [0.18814039, 0.18189439, 0.18901438, ..., 0.19213791, 0.1953843 ,
        0.2033501 ]], dtype=float32)

In [194]:
nombre = 'Bass'
now = datetime.now().strftime("%Y-%m-%d-%H_%M:%S")
"example_a_{0}_{1}".format(nombre,now)

'example_a_Bass_2018-08-23-05_23:04'

In [127]:
bb[0].reshape(-1,78,2).shape

(10, 78, 2)

In [14]:
!python /home/inquisidor/Desktop/incognito/proyecto-beatles/mergemidis.py

Traceback (most recent call last):
  File "/home/inquisidor/Desktop/incognito/proyecto-beatles/mergemidis.py", line 4, in <module>
    from csv_filter_track import csv_to_midi
  File "/home/inquisidor/Desktop/incognito/proyecto-beatles/csv_filter_track.py", line 8, in <module>
    assert(os.path.exists('./csv')),'Debe tener carpeta de csv procesados en working space'
AssertionError: Debe tener carpeta de csv procesados en working space
